In [1]:
import pandas as pd
import re, random, string, time
import sqlite3

In [2]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

In [3]:
prompt = pd.read_csv("../Data/Detect AI Generated Text/train_prompts.csv")
prompt

,prompt_id,prompt_name,instructions,source_text
0,0,Car-free cities,Write an explanatory essay to inform fellow ci...,"# In German Suburb, Life Goes On Without Cars ..."
1,1,Does the electoral college work?,Write a letter to your state senator in which ...,# What Is the Electoral College? by the Office...


In [4]:
raw_student_text = pd.read_csv('../Data/Detect AI Generated Text/train_essays.csv')
raw_generated_text = pd.read_csv('../Data/Detect AI Generated Text/ai_generated_train_essays.csv')
raw_generated_text_gpt4 = pd.read_csv('../Data/Detect AI Generated Text/ai_generated_train_essays_gpt-4.csv')
raw_generated_text = pd.concat([raw_generated_text, raw_generated_text_gpt4])
raw_text = pd.concat([raw_student_text, raw_generated_text])
id_list = raw_text['id'].tolist()
raw_text

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0
...,...,...,...,...
195,47bddafc,1,"Dear Senator [Your Senator's Name],\n\nI am wr...",1
196,76e1300e,1,"Dear Senator,\n\nI am writing today to express...",1
197,05058b6b,1,"Dear Senator,\n\nI hope this letter finds you ...",1
198,1887e7ab,1,"Senator [Your Senator’s Name],\n\nI am a const...",1


In [5]:
def generate_prompt(prompt_id):
    instruction = prompt[prompt['prompt_id'] == prompt_id]['instructions'].values[0]
    source_text = prompt[prompt['prompt_id'] == prompt_id]['source_text'].values[0]

    #Remove any '\n\n' plus the following number from the source text. For example, '\n\n1', '\n\n2', '\n\n13', etc.
    source_text = re.sub(r'\n\n\d+', '\n\n', source_text)

    #Remove any '\n\n' plus the following number from the source text. For example, '1\n\n', '2\n\n', '\n\n13', etc.
    source_text = re.sub(r'\d+\n\n', '\n\n', source_text)

    #Remove any '.\n\n' from the source text.
    source_text = re.sub(r'\.\n\n', '. \n\n', source_text)

    return instruction, source_text

In [6]:
# Generate SQL database to store generated essays as they are generated to avoid losing progress
conn = sqlite3.connect('../Data/Detect AI Generated Text/generated_essays.db')
c = conn.cursor()

# Create table if it doesn't exist
c.execute('''CREATE TABLE IF NOT EXISTS essays
             (id text,
              prompt_id integer, 
              text text,
              generated integer,
              model text)''')

# Commit changes
conn.commit()

# Close connection
conn.close()

In [7]:
# grab the essays that have already been generated during previous runs
conn = sqlite3.connect('../Data/Detect AI Generated Text/generated_essays.db')
c = conn.cursor()
c.execute('SELECT * FROM essays')
generated_essays = pd.DataFrame(c.fetchall(), columns=['id', 'prompt_id', 'text', 'generated', 'model'])
conn.close()
essays = pd.concat([raw_student_text, generated_essays])

In [8]:
def store_essay(prompt_id, text, model_id):
    prompt_id = int(prompt_id)
    #Generate a random id up to 8 characters long and check if it already exists in the list of ids.
    #If it does, generate another id and check again. Repeat until a unique id is generated.
    id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    while id in id_list:
        id = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    id_list.append(id)

    # Connect to database
    conn = sqlite3.connect('../Data/Detect AI Generated Text/generated_essays.db')
    c = conn.cursor()

    # Insert data into table
    c.execute("INSERT INTO essays VALUES (?, ?, ?, ?, ?)", (id, prompt_id, text, 1, model_id))

    # Commit changes
    conn.commit()

    # Close connection
    conn.close()

In [15]:
def generate_essay(model_id):
    for i in range(0, 2):
        instruction, source = generate_prompt(i)
        text = instruction + r'\n\n' + source
        message=[{"role": "user", "content": text}]
        j = 0
        while j < 250:
            output = client.chat.completions.create(
                                          model="local-model", # this field is currently unused
                                          messages=message,
                                          max_tokens=4096,
                                          temperature=1.8)
            essay_len = output.usage.completion_tokens
            if essay_len > 400 & essay_len < 1360:
                essay = output.choices[0].message.content
                store_essay(i, essay, model_id=model_id)
                j += 1
            time.sleep(3) # sleep for 3 seconds to avoid accessing the API too quickly

# CatPPT model by andrijdavid
https://huggingface.co/andrijdavid/CatPPT-base-GGUF

In [10]:
generate_essay('catPPT')

# Dolphin 2 model by TheBloke
https://huggingface.co/TheBloke/dolphin-2.6-mistral-7B-dpo-GGUF

In [11]:
generate_essay('dolphin')

# Mistral 7B Instruct model by TheBloke
https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF

In [12]:
generate_essay('mistral')